# Using Custom Artifact Visualizations with InteractiveContext

## How InteractiveContext Visualizes Artifacts

You call `InteractiveContext.show(component.outputs['artifact_name'])`.

`InteractiveContext` checks whether there's a visualization for the artifact in its visualization registry. If one is available, it calls the visualization's `display` method. 
   ```python
   @notebook_utils.requires_ipython
   def show(self, item: object) -> None:
      """Show the given object in an IPython notebook display."""
      from IPython.core.display import display  # pylint: disable=g-import-not-at-top
      from IPython.core.display import HTML  # pylint: disable=g-import-not-at-top
      if isinstance(item, types.Channel):
        channel = item
        artifacts = channel.get()
        for artifact in artifacts:
          artifact_heading = 'Artifact at %s' % html.escape(artifact.uri)
          display(HTML('<b>%s</b><br/><br/>' % artifact_heading))
          visualization = visualizations.get_registry().get_visualization(
              artifact.type_name)
          if visualization:
            visualization.display(artifact)
      else:
        display(item)
   ```

Default visualizations are defined in tfx.orchestration.experimental.interactive.standard_visualizations, and they're registerd in the `InteractiveContext`'s constructor.

## How to Create Custom Visualizations

1. Subclass the `ArtifactVisualization` abstract base class.
2. Override the `ARTIFACT_TYPE` property with the type of artifact the visualization applies to.
3. Override the `display` method to read relevant content from your artifact's URI and to render the content.
4. Add the visualization to the visulaization registry.

## Let's Walk Through an Example!

We'll take a look at the first three training examples produced by running `ExampleGen`. We'll start with the method used in the [TFX Keras Component tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras), then we'll refactor that code to create a custom visualization.

In [9]:
# Run the component
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Chicago Elite Cab Corp. (Chicago Carriag"
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 12.449999809265137
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      bytes_list {
        value: "Credit Card"
      }
    }
  }
  feature {
    key: "pickup_census_tract"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "pickup_community_area"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
   

2022-10-19 10:49:33.985606: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
from tfx.orchestration.experimental.interactive import visualizations
from tfx.types import artifact,  standard_artifacts

class ExampleVisualization(visualizations.ArtifactVisualization):
  ARTIFACT_TYPE = standard_artifacts.Examples

  def display(self, artifact: artifact.Artifact, count=3):
    from IPython.display import display, HTML
    
    # Get the URI of the output artifact representing the training examples, which is a directory
    train_uri = os.path.join(artifact.uri, 'Split-train')

    # Get the list of files in this directory (all compressed TFRecord files)
    tfrecord_filenames = [os.path.join(train_uri, name)
                          for name in os.listdir(train_uri)]

    # Create a `TFRecordDataset` to read these files
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

    # Iterate over the first count records and decode them.
    for tfrecord in dataset.take(count):
      serialized_example = tfrecord.numpy()
      example = tf.train.Example()
      example.ParseFromString(serialized_example)
      display(HTML(f'<pre>{example}</pre>'))

visualizations.get_registry().register(ExampleVisualization)

context.show(example_gen.outputs['examples'])

We can use the same visualization we just created for transformed examples as well:

In [33]:
context.show(transform.outputs['transformed_examples'])

**Snippets from the TFX Keras Component tutorial and the InteractiveContext.show() definition are under Copyright 2021 by The TensorFlow Authors and licensed under the Apache License, Version 2.0.**